<a href="https://colab.research.google.com/github/KevinJayne2023/agent_recruiter_screening_langgraph/blob/main/agent_recruiter_screening_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recruiter Screening Notes — LangChain + LangGraph

Paste raw screening-call notes → get polished **Screening Notes** (Markdown) and **5 job title suggestions** with rationales.

**What this notebook does**
- Extracts structured data from messy notes (name, experience, skills, comp, flags...)
- Normalizes skills
- Suggests 5 job titles with brief rationales
- Produces a final Markdown summary + JSON payload you can paste to your ATS/CRM

**Stack**: LangChain, LangGraph, OpenAI (or swap model), Pydantic, RapidFuzz


## 1) Install Dependencies

In [1]:
!pip -q install "langchain>=0.2.10" "langgraph>=0.2.20" "langchain-openai>=0.1.20" "pydantic>=2.7.0" "rapidfuzz>=3.7.0" "python-dotenv>=1.0.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 11.7 MB/s eta 0:00:00


## 2) Configure API key

In [2]:
import os, getpass
if not os.environ.get("OPENAI_API_KEY"):
    try:
        os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")
    except Exception as e:
        print("Set it manually: os.environ['OPENAI_API_KEY'] = ...")
print("API key set:", bool(os.environ.get("OPENAI_API_KEY")))

OPENAI_API_KEY··········
API key set: True


## 3) Define models, helpers, and ontology

In [18]:
from __future__ import annotations
from typing import List, Optional, Dict, Any, TypedDict
from dataclasses import dataclass
from pydantic import BaseModel, Field
import os, re, json
from rapidfuzz import fuzz
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END
from IPython.display import Markdown, display

class ExperienceItem(BaseModel):
    title: Optional[str] = None
    company: Optional[str] = None
    start: Optional[str] = None
    end: Optional[str] = None
    highlights: List[str] = Field(default_factory=list)

class ScreeningExtraction(BaseModel):
    candidate_name: Optional[str] = None
    current_title: Optional[str] = None
    current_company: Optional[str] = None
    years_experience: Optional[float] = None
    location: Optional[str] = None
    work_authorization: Optional[str] = None
    remote_on_site_pref: Optional[str] = None
    comp_current: Optional[str] = None
    comp_expected: Optional[str] = None
    notice_period: Optional[str] = None
    earliest_start_date: Optional[str] = None
    role_preferences: List[str] = Field(default_factory=list)
    top_skills: List[str] = Field(default_factory=list)
    tools_stack: List[str] = Field(default_factory=list)
    industries: List[str] = Field(default_factory=list)
    risk_flags: List[str] = Field(default_factory=list)
    notable_quotes: List[str] = Field(default_factory=list)
    experience: List[ExperienceItem] = Field(default_factory=list)
    summary_paragraph: Optional[str] = None

class TitleSuggestion(BaseModel):
    title: str
    rationale: str
    confidence: float = Field(ge=0, le=1)

class ScreenState(TypedDict, total=False):
    raw_notes: str
    cleaned_notes: str
    extracted: ScreeningExtraction
    normalized_skills: List[str]
    title_suggestions: List[TitleSuggestion]
    formatted_markdown: str
    json_payload: Dict[str, Any]

def make_llm(model: str = "gpt-4o-mini", temperature: float = 0.1):
    return ChatOpenAI(model=model, temperature=temperature)

def normalize_text(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

# --- Canonical skills ---
CANONICAL_SKILLS = {
    "python": ["py", "python3"],
    "java": [],
    "javascript": ["js", "ecmascript"],
    "typescript": ["ts"],
    "react": ["reactjs", "react.js"],
    "node": ["nodejs", "node.js"],
    "sql": [],
    "aws": ["amazon web services"],
    "gcp": ["google cloud"],
    "azure": [],
    "docker": [],
    "kubernetes": ["k8s", "kube"],
    "spark": ["pyspark"],
    "hadoop": [],
    "airflow": [],
    "terraform": [],
    "ci/cd": ["cicd", "ci cd"],
    # data/ml
    "pandas": [],
    "numpy": [],
    "pytorch": [],
    "tensorflow": ["tf", "tfx"],
    "sklearn": ["scikit-learn"],
    "mlops": ["sagemaker", "vertex ai", "mlflow"],
    "nlp": ["natural language processing"],
    "computer vision": ["cv"],
    "llm": ["large language model", "gpt", "llama"],
    # RAG / LLM tooling
    "langchain": [],
    "langgraph": [],
    "prompt engineering": ["prompting", "system prompt", "few-shot"],
    "rag": ["retrieval-augmented generation", "retrieval"],
    "faiss": [],
    "pgvector": [],
    "pinecone": [],
    "openai": [],
    "anthropic": [],
    # analytics / data tools
    "dbt": [],
    # sre/devops
    "prometheus": [],
    "grafana": [],
    # qa/security
    "selenium": [],
    "cypress": [],
    "pytest": [],
    "security": [],
    "iam": [],
    "kms": [],
    "siem": [],
}

TITLE_ONTOLOGY = [
    "Software Engineer (Backend)",
    "Software Engineer (Frontend)",
    "Full-Stack Engineer",
    "Data Engineer",
    "Machine Learning Engineer",
    "MLOps Engineer",
    "Data Scientist",
    "Analytics Engineer",
    "DevOps Engineer",
    "Site Reliability Engineer (SRE)",
    "QA Automation Engineer",
    "Security Engineer",
    "Solutions Architect",
    "Product Manager (Technical)",
    "AI Engineer",
    "Prompt Engineer",
]

TITLE_KEYWORDS = {
    "Software Engineer (Backend)": ["python", "java", "node", "sql", "aws", "gcp", "docker", "kubernetes"],
    "Software Engineer (Frontend)": ["javascript", "typescript", "react"],
    "Full-Stack Engineer": ["react", "node", "python", "typescript"],
    "Data Engineer": ["spark", "airflow", "python", "sql", "hadoop", "aws"],
    "Machine Learning Engineer": ["pytorch", "tensorflow", "sklearn", "mlops", "llm", "nlp", "computer vision"],
    "MLOps Engineer": ["mlops", "kubernetes", "docker", "aws", "gcp", "mlflow", "sagemaker", "vertex ai"],
    "Data Scientist": ["python", "pandas", "numpy", "sklearn", "nlp", "llm"],
    "Analytics Engineer": ["sql", "dbt", "warehouse", "etl", "airflow"],
    "DevOps Engineer": ["aws", "gcp", "azure", "kubernetes", "docker", "terraform", "ci/cd"],
    "Site Reliability Engineer (SRE)": ["kubernetes", "observability", "prometheus", "grafana", "incident"],
    "QA Automation Engineer": ["selenium", "cypress", "pytest"],
    "Security Engineer": ["security", "iam", "kms", "siem"],
    "Solutions Architect": ["pre-sales", "solutions", "customer", "cloud", "architecture"],
    "Product Manager (Technical)": ["product", "requirements", "backlog", "roadmap", "stakeholder"],
    "AI Engineer": [
        "llm", "rag", "langchain", "langgraph", "embeddings", "vector", "faiss", "pgvector",
        "pinecone", "openai", "anthropic", "function calling", "tools", "agents", "retrieval",
        "guardrails", "evaluation", "observability", "python", "pydantic"
    ],
    "Prompt Engineer": [
        "prompt", "few-shot", "system prompt", "templates", "structured output", "json",
        "function calling", "tools", "eval", "evaluation", "guardrail", "jailbreak",
        "injection", "rag", "retrieval", "langchain", "langgraph", "rubrics", "A/B"
    ],
}

def normalize_skills(skills: List[str]) -> List[str]:
    canon = set()
    for raw in skills:
        if not raw:
            continue
        low = raw.lower().strip()
        hit = False
        for key, aliases in CANONICAL_SKILLS.items():
            if low == key or low in aliases:
                canon.add(key)
                hit = True
                break
        if hit:
            continue
        best = None
        best_score = 0
        for key, aliases in CANONICAL_SKILLS.items():
            for term in [key] + aliases:
                score = fuzz.partial_ratio(low, term)
                if score > best_score:
                    best, best_score = key, score
        if best and best_score >= 85:
            canon.add(best)
        else:
            canon.add(raw)
    return sorted(canon)

def score_titles(skills: List[str], text_blob: str) -> List[tuple[str, float]]:
    low_text = text_blob.lower()
    scores = []
    skillset = set([s.lower() for s in skills])
    for title in TITLE_ONTOLOGY:
        kws = TITLE_KEYWORDS.get(title, [])
        hit_skill = len(skillset.intersection(set(kws)))
        hit_text = sum(1 for k in kws if k in low_text)
        score = hit_skill * 1.0 + hit_text * 0.5
        scores.append((title, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores


## 4) Build the LangGraph and nodes

In [19]:
def preprocess_node(state: ScreenState) -> ScreenState:
    notes = state.get("raw_notes", "")
    cleaned = normalize_text(notes)
    return {"cleaned_notes": cleaned}

def extract_node(state: ScreenState) -> ScreenState:
    llm = make_llm()
    schema_json = ScreeningExtraction.schema_json(indent=2)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a meticulous recruiter’s assistant. Extract structured fields from the call notes. "
         "If a field is unknown, leave it null or empty. Do not invent facts."),
        ("human",
         "Call notes:\n\n{notes}\n\n"
         "Return ONLY a JSON object matching this Pydantic schema:\n{schema}")
    ])

    chain = prompt | llm.with_structured_output(ScreeningExtraction)
    extracted: ScreeningExtraction = chain.invoke({
        "notes": state["cleaned_notes"],
        "schema": schema_json
    })
    return {"extracted": extracted}


def normalize_skills_node(state: ScreenState) -> ScreenState:
    extracted: ScreeningExtraction = state["extracted"]
    combined = list({*(extracted.top_skills or []), *(extracted.tools_stack or [])})
    norm = normalize_skills(combined)
    return {"normalized_skills": norm}

def suggest_titles_node(state: ScreenState) -> ScreenState:
    extracted: ScreeningExtraction = state["extracted"]
    skills = state.get("normalized_skills", [])

    text_blob = " ".join([
        state.get("cleaned_notes", ""),
        " ".join(skills),
        extracted.summary_paragraph or "",
        (extracted.current_title or "") + " " + " ".join(extracted.role_preferences or [])
    ])
    base_scores = score_titles(skills, text_blob)
    top8 = [t for t, s in base_scores[:8] if s > 0] or [t for t, _ in base_scores[:8]]

    llm = make_llm(temperature=0.2)
    context_json = extracted.json()

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You propose 5 job titles a candidate fits and explain why. Calibrate confidence 0.6–0.95."),
        ("human",
         "Candidate context (JSON):\n{context_json}\n\n"
         "Normalized skills: {skills}\n"
         "Title candidates to choose from: {top8}\n\n"
         "Return a JSON array of 5 objects with keys: title, rationale, confidence.")
    ])

    parser = StrOutputParser()
    raw = (prompt | llm | parser).invoke({
        "context_json": context_json,
        "skills": skills,
        "top8": top8
    })

    try:
        parsed = json.loads(raw)
        suggestions = [TitleSuggestion(**d) for d in parsed][:5]
    except Exception:
        suggestions = [
            TitleSuggestion(title=t, rationale="Matches skill keywords and experience signals.", confidence=0.7)
            for t, _ in base_scores[:5]
        ]
    return {"title_suggestions": suggestions}

def format_markdown_node(state: ScreenState) -> ScreenState:
    e: ScreeningExtraction = state["extracted"]
    titles: List[TitleSuggestion] = state["title_suggestions"]
    skills = ", ".join(state.get("normalized_skills", []))

    md = []
    md.append(f"# Screening Notes — {e.candidate_name or 'Candidate'}")
    md.append("")
    md.append("## Overview")
    md.append(e.summary_paragraph or "—")
    md.append("")
    md.append("## Snapshot")
    md.append(f"- **Current Title/Company:** {e.current_title or '—'} @ {e.current_company or '—'}")
    md.append(f"- **Location:** {e.location or '—'}")
    md.append(f"- **Years Experience:** {e.years_experience if e.years_experience is not None else '—'}")
    md.append(f"- **Work Authorization:** {e.work_authorization or '—'}")
    md.append(f"- **Work Preference:** {e.remote_on_site_pref or '—'}")
    md.append(f"- **Comp (Current):** {e.comp_current or '—'}")
    md.append(f"- **Comp (Expected):** {e.comp_expected or '—'}")
    md.append(f"- **Notice / Availability:** {e.notice_period or '—'} / {e.earliest_start_date or '—'}")
    md.append("")
    md.append("## Role Preferences")
    md.append(("; ".join(e.role_preferences)) if e.role_preferences else "—")
    md.append("")
    md.append("## Skills & Stack")
    md.append(skills or "—")
    md.append("")
    if e.experience:
        md.append("## Experience Highlights")
        for item in e.experience:
            line = f"- **{item.title or '—'}**, {item.company or '—'}"
            if item.start or item.end:
                line += f" ({item.start or ''}–{item.end or ''})"
            md.append(line)
            for h in (item.highlights or []):
                md.append(f"  - {h}")
        md.append("")
    if e.risk_flags:
        md.append("## Risks / Concerns")
        for r in e.risk_flags:
            md.append(f"- {r}")
        md.append("")
    if e.notable_quotes:
        md.append("## Notable Quotes")
        for q in e.notable_quotes:
            md.append(f"> {q}")
        md.append("")
    md.append("## Suggested Job Titles (Top 5)")
    for i, t in enumerate(titles[:5], 1):
        md.append(f"{i}. **{t.title}** — {t.rationale} _(confidence: {t.confidence:.2f})_")

    payload = {
        "extracted": json.loads(e.model_dump_json()),
        "normalized_skills": state.get("normalized_skills", []),
        "title_suggestions": [t.model_dump() for t in titles[:5]],
    }
    return {"formatted_markdown": "\n".join(md), "json_payload": payload}

def build_graph():
    sg = StateGraph(ScreenState)
    sg.add_node("preprocess", preprocess_node)
    sg.add_node("extract", extract_node)
    sg.add_node("normalize_skills", normalize_skills_node)
    sg.add_node("suggest_titles", suggest_titles_node)
    sg.add_node("format_markdown", format_markdown_node)
    sg.set_entry_point("preprocess")
    sg.add_edge("preprocess", "extract")
    sg.add_edge("extract", "normalize_skills")
    sg.add_edge("normalize_skills", "suggest_titles")
    sg.add_edge("suggest_titles", "format_markdown")
    sg.add_edge("format_markdown", END)
    return sg.compile()

## 5) Paste your call notes and run
Enter redacted notes below

In [20]:
RAW_NOTES = """
Add notes here
""".strip()
print(RAW_NOTES[:1000] + ("..." if len(RAW_NOTES)>1000 else ""))

Add notes here


## 6) Run the graph -> View results (Markdown + JSON)

In [21]:
graph = build_graph()
out = graph.invoke({"raw_notes": RAW_NOTES})
md = out["formatted_markdown"]
payload = out["json_payload"]
display(Markdown(md))
print("\n--- JSON payload ---\n")
print(json.dumps(payload, indent=2))

/tmp/ipython-input-770463688.py:8: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  schema_json = ScreeningExtraction.schema_json(indent=2)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## 7) Save to files
Saves Markdown and JSON to `/content/`. You can also upload them to your ATS.

In [ ]:
from pathlib import Path
Path('/content').mkdir(parents=True, exist_ok=True)
md_path = '/content/screening_notes.md'
json_path = '/content/screening_payload.json'
with open(md_path, 'w', encoding='utf-8') as f:
    f.write(md)
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(payload, f, indent=2)
print('Saved:', md_path)
print('Saved:', json_path)

## Install Gradio

In [ ]:
!pip -q install gradio>=4.44.0


## Gradio UI for Screening Notes

In [ ]:
# --- Gradio UI for Screening Notes (optional file saving) ---
import os, json, tempfile
import gradio as gr

DEFAULT_SAMPLE = (
    "Candidate: Alex Kim. Based in Austin, open to hybrid. US Citizen.\n"
    "6.5 years experience. Current: Senior Data Engineer @ FinTechCo (Python, Spark, Airflow, AWS, Terraform).\n"
    "Built streaming ETL on Kinesis; led migration to Snowflake; strong SQL. Interested in ML platform + LLM/RAG work.\n"
    "Current comp: 165k base + 10% bonus. Target 180–200k total. Notice: 2 weeks.\n"
    "Prefers IC roles, hands-on; not interested in pure analytics. Led a squad of 3.\n"
    "Tools: GitHub Actions, Docker, dbt, LangChain, LangGraph. Risk: limited real-time MLOps, more batch.\n"
    "Quote: 'I love making pipelines boring and reliable.'"
)

def run_screening(notes: str, enable_downloads: bool):
    if not notes or not notes.strip():
        return ("Please paste screening notes.", {}, gr.update(visible=False), gr.update(visible=False))

    out = graph.invoke({"raw_notes": notes})
    md = out["formatted_markdown"]
    payload = out["json_payload"]

    if enable_downloads:
        # temp files user can download
        md_fd, md_path = tempfile.mkstemp(suffix=".md")
        with os.fdopen(md_fd, "w", encoding="utf-8") as f:
            f.write(md)
        json_fd, json_path = tempfile.mkstemp(suffix=".json")
        with os.fdopen(json_fd, "w", encoding="utf-8") as f:
            json.dump(payload, f, indent=2)
        return (md, payload, gr.update(value=md_path, visible=True), gr.update(value=json_path, visible=True))
    else:
        return (md, payload, gr.update(value=None, visible=False), gr.update(value=None, visible=False))

with gr.Blocks(title="Screening Notes (LangGraph)") as demo:
    gr.Markdown(
        """
        # Screening Notes — LangChain + LangGraph
        Paste your call notes below. Click **Generate** to produce formatted notes and title suggestions.
        Use the toggle if you want downloadable files (optional).
        """
    )
    with gr.Row():
        notes = gr.Textbox(
            label="Paste screening notes",
            value=DEFAULT_SAMPLE,
            lines=16,
            show_copy_button=True,
            placeholder="Paste your raw call notes here..."
        )
    with gr.Row():
        enable_downloads = gr.Checkbox(label="Enable downloads (save temp files)", value=False)
    btn = gr.Button("Generate", variant="primary")

    md_out = gr.Markdown(label="Formatted Screening Notes")
    json_out = gr.JSON(label="JSON Payload")

    with gr.Row():
        md_file = gr.File(label="Download Markdown", visible=False)
        json_file = gr.File(label="Download JSON", visible=False)

    btn.click(
        fn=run_screening,
        inputs=[notes, enable_downloads],
        outputs=[md_out, json_out, md_file, json_file]
    )

demo.launch(share=True)
